In [41]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder,LabelEncoder

import pickle

In [42]:
# load the dataset

data = pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [43]:
## preprocess the data

# drop irrelevent columns
data = data.drop(['RowNumber','CustomerId','Surname'],axis=1)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [44]:
# Encode Categorical Variables

label_encoder = LabelEncoder()

cat_cols = [col for col in data.columns if data[col].dtype=='O']

# cat_cols =['Geography']

# label_encoder.fit_transform(data[['Geography','Gender']])  #give error

## works fine to transform but overrides the label_encoder of previous col
# data[cat_cols]= data[cat_cols].apply(label_encoder.fit_transform)

## so the inverse_transform of previous columns can't be done 
# label_encoder.inverse_transform(data['Geography'])


# label_encoder.classes_
# label_encoder.transform(label_encoder.classes_)
# label_encoder.inverse_transform([1,0])


In [45]:
# label_object = {}
# categorical_columns = ['Gender','Geography']
# for col in categorical_columns:
# --- labelencoder = LabelEncoder()
# --- labelencoder.fit(df[col])
# --- df[col] = labelencoder.fit_transform(df[col])
# --- label_object[col] = labelencoder

# label_object['Gender'].inverse_transform(df['Gender'])

In [46]:
label_encoder_gender = LabelEncoder()

data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [47]:
# one-hot-encode Geography column

oh_encoder_geo = OneHotEncoder(sparse_output=False,handle_unknown='ignore')
geo_encoded = oh_encoder_geo.fit_transform(data[['Geography']])

In [48]:
oh_encoder_geo.get_feature_names_out()

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [49]:
geo_encoded_df =pd.DataFrame(geo_encoded,columns=oh_encoder_geo.get_feature_names_out())
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [50]:
data = pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)

In [51]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [52]:
## divide the data into dependent and independent features
X = data.drop('Exited',axis=1)
y = data['Exited']

## split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

print(X_train.shape,y_train.shape)
print(X_test.shape, y_test.shape)


(8000, 12) (8000,)
(2000, 12) (2000,)


In [53]:
# Scale the features

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [54]:
scaler.feature_names_in_

array(['CreditScore', 'Gender', 'Age', 'Tenure', 'Balance',
       'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary',
       'Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [55]:
X_train

array([[ 0.35649971,  0.91324755, -0.6557859 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.91324755,  0.29493847, ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213,  0.91324755, -1.41636539, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  0.91324755,  1.15059039, ..., -0.99850112,
         1.72572313, -0.57638802]])

In [56]:
## save the encoders and scalers
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(oh_encoder_geo,file)

with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

ANN Implementation

In [57]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [58]:
# build Ann model
input_shape = (X_train.shape[1],)

model= Sequential([
    Dense(units=64,
          input_shape=input_shape,
          name='layer-0', 
          activation='relu'),  #HL1 connected with input layer
    Dense(units=32,
          name='layer-1',
          activation='relu'),  #HL2 
    Dense(units=1,
          name='output-layer', 
          activation='sigmoid')  #output layer
])


model.summary()

d:\Github\churn_prediction\my_env\Lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ layer-0 (Dense)                 │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer-1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output-layer (Dense)            │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [59]:
opt = tf.keras.optimizers.Adam(learning_rate=0.01)
loss = tf.keras.losses.BinaryCrossentropy()

In [60]:
# compile the model

model.compile(optimizer=opt,   #to customize learning_rate
            #   optimizer="adam",
            # loss =loss,
              loss='binary_crossentropy',
              metrics=['accuracy'] )

In [61]:
## Set up the Tensorboard (to capture and visualize the logs) (after training)

log_dir = "logs/fit/" + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')

tensorflow_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [62]:
## set up Early Stopping

early_stopping_callback = EarlyStopping(monitor='val_loss',
                                        patience=5,
                                        restore_best_weights=True)



In [63]:
# Train the Model

history = model.fit(X_train, y_train,
                validation_data =(X_test, y_test),
                epochs=100,
                callbacks=[tensorflow_callback,early_stopping_callback])

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8078 - loss: 0.4374 - val_accuracy: 0.8525 - val_loss: 0.3545
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8581 - loss: 0.3443 - val_accuracy: 0.8605 - val_loss: 0.3619
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8524 - loss: 0.3501 - val_accuracy: 0.8560 - val_loss: 0.3486
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8606 - loss: 0.3419 - val_accuracy: 0.8535 - val_loss: 0.3529
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8577 - loss: 0.3383 - val_accuracy: 0.8580 - val_loss: 0.3421
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8574 - loss: 0.3436 - val_accuracy: 0.8625 - val_loss: 0.3392
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8633 - loss: 0.3336 - val_accuracy: 0.8600 - val_loss: 0.3418
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8664 - loss: 0.3233 - val_accu

In [64]:
## Save the model
# h5 is compatable with keras

model.save('model.h5') 

In [65]:
log_dir

'logs/fit/20250613-205527'

In [66]:
# Load Tensorboard Extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [67]:
%reload_ext tensorboard

In [68]:
# %tensorboard --logdir logs/fit/ 